### Operating on Data in Pandas

In [1]:
import pandas as pd
import numpy as np

Two additional indexing examples:  
- use of -1
- index duplicates

In [ ]:
ser = pd.Series(np.arange(3.))
ser

In [ ]:
ser[-1] # generates an error in the case of integer indexing

In [ ]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2

In [ ]:
ser2[-1]  # not a problem with non-integer indexing

Indexes with duplicate labels

In [ ]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

In [ ]:
obj.index.is_unique

In [ ]:
obj['c']

In [ ]:
obj['a'] # index duplicates will return a series

In [ ]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

In [ ]:
df.loc['b']

Pandas is designed to work with NumPy, so any NumPy ufunc will work on Pandas Series and DataFrame objects. 

For unary operations like negation and trigonometric functions, these ufuncs preserve index and column labels in the output.


For binary operations such as addition and multiplication, Pandas will automatically align indices when passing the objects to the ufunc. 


In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4)) # create a series object
ser

In [ ]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D']) # create a Dataframe object
df

In [ ]:
df.axes

If we apply a NumPy ufunc on either of these objects, the result will be another Pandas object with the indices preserved:

In [ ]:
np.exp(ser) # exponential function - e^x where e is Euler's number


In [ ]:
np.sin(df * np.pi / 4)

For binary operations on two Series or DataFrame objects, Pandas will align indices in the process of performing the operation. This is very convenient when working with incomplete data.

In [ ]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
print(population)
print(area)

In [ ]:
population / area

The resulting array contains the union of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

In [ ]:
area.index | population.index

Any item for which one or the other does not have an entry is marked with NaN, which is how Pandas marks missing data. 

This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

In [ ]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
print(A)
print(B)
A + B

If using NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators. For example, calling A.add(B) is equivalent to calling A + B, but allows optional explicit specification of the fill value for any elements in A or B that might be missing:

In [ ]:
A.add(B, fill_value=0)

#### Index alignment in DataFrame
A similar type of alignment takes place for both columns and indices when performing operations on DataFrames:

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

In [ ]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

In [ ]:
A + B

Notice that indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted

As was the case with Series, we can use the associated object's arithmetic method and pass any desired fill_value to be used in place of missing entries. We can fill with the mean of all values in A (computed by first stacking the rows of A):

In [ ]:
fill = A.stack().mean()
fill

In [ ]:
A.add(B, fill_value=fill)

The following table lists Python operators and their equivalent Pandas object methods:

Python Operator	Pandas Method(s)

    +	add()
    -	sub(), subtract()
    *	mul(), multiply()
    /	truediv(), div(), divide()
    //	floordiv()
    %	mod()
    **	pow()

#### Ufuncs: Operations Between DataFrame and Series

When performing operations between a DataFrame and a Series, the index and column alignment is similarly maintained. Operations between a DataFrame and a Series are similar to operations between a two-dimensional and one-dimensional NumPy array.

Consider one common operation, where we find the difference of a two-dimensional array and one of its rows:

In [ ]:
A = rng.randint(10, size=(3, 4))
A

In [ ]:
A - A[0]

According to NumPy's broadcasting rules, subtraction between a two-dimensional array and one of its rows is applied row-wise.

In Pandas, the convention similarly operates row-wise by default:

In [ ]:
df = pd.DataFrame(A, columns=list('QRST'))
df

In [ ]:
df - df.iloc[0]

If you would instead like to operate column-wise, you can use the object methods mentioned earlier, while specifying the axis keyword:

In [ ]:
df

In [ ]:
df.subtract(df['R'], axis=0)

Note that these DataFrame/Series operations, like the operations discussed above, will automatically align indices between the two elements:

In [ ]:
halfrow = df.iloc[0, ::2]
halfrow

In [ ]:
df - halfrow

Subtraction is performed once for each row.

In [ ]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

In [ ]:
series = frame.iloc[0]
series

By default arithmetic betweeen a Dataframe and Series matches the index of the Series on the Dataframe, broadcastign down the rows.

In [ ]:
frame - series

In [ ]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
series2

In [ ]:
frame

In [ ]:
frame + series2

If you want to broadcast over the columns, matching on rows, you have to use one of the arithmetic methods.

In [ ]:
series3 = frame['d']
series3

In [ ]:
frame

In [ ]:
frame.sub(series3, axis='index')

This preservation and alignment of indices and columns means that operations on data in Pandas will always maintain the data context, which prevents the types of errors that might come up when working with heterogeneous and/or misaligned data in raw NumPy arrays.

### Function Application and Mapping
Numpy Unfuncs work on pandas objects

In [ ]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame


In [ ]:
np.abs(frame)

Apply a function on a one -dimensional array using the DataFrame apply method.

The function f computes the difference between the max and min of a Series.

It is invoked on each column in the frame by apply.

In [ ]:
f = lambda x: x.max() - x.min()

In [ ]:
frame.apply(f)

Using axis = 'columns" applies the function to the rows

In [ ]:
frame.apply(f, axis='columns')

In [ ]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])


In [ ]:
frame.apply(f)

### Handling Missing Data

Real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of data missing. Additionally, different data sources may indicate missing data in different ways.

#### Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, these are one of two strategies: using a mask that globally indicates missing values, or choosing a sentinel value that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value is typically some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

Each of these approaches has trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

### Missing Data in Pandas
The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas uses as sentinels two already-existing Python null values: the special floating-point NaN value, and the Python None object. 

None, is a Python object that is often used for missing data in Python code. Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type 'object' (i.e., arrays of Python objects):

In [ ]:
vals1 = np.array([1, None, 3, 4])
vals1

In [ ]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E4, dtype=dtype).sum()
    print()

The use of Python objects in an array also means that if you perform aggregations like sum() or min() across an array with a None value, you will generally get an error:

In [ ]:
vals1.sum()

The NaN (acronym for Not a Number) missing data representation is different.
It is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [ ]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code. 

An issues with NaN  - it is a bit like a data virus–it infects any other object it touches. Regardless of the operation, the result of arithmetic with NaN will be another NaN.

In [ ]:
1 + np.nan

In [ ]:
0 *  np.nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but are not always useful:

In [ ]:
vals2.sum(), vals2.min(), vals2.max()

NumPy does provide some special aggregations that will ignore these missing values:

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

NaN is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [ ]:
pd.Series([1, np.nan, 2, None])

Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the NA:

In [ ]:
x = pd.Series(range(2), dtype=int)
x

In [ ]:
x[0] = None
x

In addition to casting the integer array to floating point, Pandas automatically converts the None to a NaN value. 

Pandas has several useful methods for detecting, removing, and replacing null values. They are:

- isnull(): Generate a boolean mask indicating missing values
- notnull(): Opposite of isnull()
- dropna(): Return a filtered version of the data
- fillna(): Return a copy of the data with missing values filled or imputed

Two useful methods for detecting null data are: isnull() and notnull(). Either one will return a Boolean mask over the data. For example:

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
data

In [ ]:
data.isnull()

In [ ]:
data[data.notnull()]

There are the convenience methods, dropna() (which removes NA values) and fillna() (which fills in NA values). 

For a Series, the result is straightforward:

In [ ]:
data.dropna()

We cannot drop single values from a DataFrame; we can only drop full rows or full columns. Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.

By default, dropna() will drop all rows in which any null value is present:

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

In [ ]:
df.dropna()

Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value:

In [ ]:
df.dropna(axis='columns')

But this can drop some good data.

You might want to only drop rows or columns with all NA values, or a majority of NA values. 

This can be specified through the how or thresh parameters, which allow fine control of the number of nulls to allow through.

The default is how='any', such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify how='all', which will only drop rows/columns that are all null values:

In [ ]:
df[3] = np.nan
df

In [ ]:
df.dropna(axis='columns', how='all')

For finer-grained control, the thresh parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [ ]:
df.dropna(axis='rows', thresh=3)

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.

This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. 

You can do this in-place using the isnull() method as a mask, but because it is such a common operation Pandas provides the fillna() method, which returns a copy of the array with the null values replaced.

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

We can fill NA entries with a single value, such as zero:

In [ ]:
data.fillna(0)

We can specify a forward-fill to propagate the previous value forward:

In [ ]:
data.fillna(method='ffill')

For DataFrames, the options are similar, but we can also specify an axis along which the fills take place:

In [ ]:
df

In [ ]:
df.fillna(method='ffill', axis=1)

Notice that if a previous value is not available during a forward fill, the NA value remains.